- **토치텍스트(Torchtext)** : PyTorch에서 텍스트에 대한 여러 추상화 기능을 제공하는 자연어 처리 라이브러리
- 토치텍스트의 기능
    - 파일 로드(Flie Loading) : 다양한 포맷의 코퍼스 로드
    - 토큰화(Tokenization) : 단어 단위로 문장 분리
    - 단어 집합(Vocab) : 단어 집합 생성
    - 정수 인코딩(Integer encoding) : 전체 코퍼스의 단어들을 각각의 고유한 정수로 맵핑
    - 단어 벡터(Word Vector) : 단어 집합의 단어들에 고유한 임베딩 벡터를 만든다. 랜덤값으로 초기화한 값이거나 사전 훈련된 임베딩 벡터를 로드할 수도 있다.
    - 배치화(Batching) : 훈련 샘플들의 배치를 만든다. 이 과정에서 패딩(Padding) 작업도 이루어진다.


# 훈련 데이터, 테스트 데이터 분리

In [8]:
import urllib.request
import pandas as pd

In [9]:
# IMDB 리뷰 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x1c4b80074c0>)

In [10]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [11]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [12]:
train_df = df[:25000]
test_df = df[25000:]

In [13]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

# 필드 정의 (torchtext.data)

- 필드를 통해 앞으로 어떤 전처리를 할 것인지 정의한다.

In [14]:
from torchtext import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

AttributeError: module 'torchtext.data' has no attribute 'Field'

- TEXT : 실제 테스트를 위한 객체
- LABEL : 레이블 데이터를 위한 객체

- 각 인자에 대한 설명
    - sequential : 시퀀스 데이터 여부. (True가 기본값)
    -use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
    -tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
    - lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
    - batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
    - is_target : 레이블 데이터 여부. (False가 기본값)
    - fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

# 데이터셋 만들기

In [15]:
from torchtext.data import TabularDataset

ImportError: cannot import name 'TabularDataset' from 'torchtext.data' (C:\Users\hcw2l\AppData\Local\Programs\Python\Python38\lib\site-packages\torchtext\data\__init__.py)

- TabularDataset은 데이터를 불러오면 필드에서 정의한 토큰화 방법으로 토큰화를 수행하며 소문자화 같은 기본적이 전처리가 함께 이루어진다.

In [16]:
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

NameError: name 'TabularDataset' is not defined

- path : 파일이 위치한 경로.
- format : 데이터의 포맷.
- fields : 위에서 정의한 필드를 지정. 첫번째 원소는 데이터 셋 내에서 해당 필드를 호칭할 이름, 두번째 원소는 지정할 필드.
- skip_header : 데이터의 첫번째 줄은 무시.

In [ ]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

In [ ]:
print(vars(train_data[0]))

In [ ]:
# 필드 구성 확인.
print(train_data.fields.items())

# Vocabulary 만들기

In [ ]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [ ]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

In [ ]:
print(TEXT.vocab.stoi)

- 단어 집합의 크기를 10000개로 제한했지만 실제로는 0~10001번까지 총 10002개인 이유는 토치텍스트에서 임의로 특별 토큰 **<unk\>** 와 **<pad\>** 를 추가했기 때문이다.
- <unk\> : 집합에 없는 단어 표현
- <pad\> : 길이 맞추는 패딩 작업

# 토치텍스트의 데이터로더 만들기

In [ ]:
from torchtext.data import Iterator

In [ ]:
batch_size = 5

In [ ]:
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
test_loader = Iterator(dataset=test_data, batch_size = batch_size)

In [ ]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니배치

In [ ]:
print(type(batch))

- 일반적 데이터로더 : 미니 배치를 텐서로 가져온다.
- 토치텍스트 데이터로더 : torchtext.data.batch.Batch 객체를 가져온다.

In [ ]:
print(batch.text)

- 각 샘플은 더 이상 단어 시퀀스가 아니라 정수 시퀀스임을 볼 수 있다.
- 중간에는 숫자 0이 존재하는데 이는 <unk\> 토큰의 번호로 단어 집합에 포함되지 못한 단어들은 <unk\>라는 토큰으로 변환되었다.

# <pad\> 토큰이 사용되는 경우

In [ ]:
batch = next(iter(train_loader)) # 첫번째 미니배치
print(batch.text[0]) # 첫번째 미니배치 중 첫번째 샘플